In [3]:
# ! pip install impedance

import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.fft import fft, irfft
from impedance import preprocessing
from sklearn.linear_model import LinearRegression
from impedance.models.circuits import Randles, CustomCircuit
pd.set_option('mode.chained_assignment', None)

class Impedance():
    '''
    此python类根据一组阻抗数据完成一系列电化学相关分析, 主要功能有:
    (1)对阻抗数据的Nyquist图进行等效电路拟合, 计算R_t和C_p的值;
    (2)对阻抗数据进行修正;
    (3)对数据进行反响傅立叶和傅立叶变化,求ds/fc
    '''
    
    def __init__(self, file_path):
        self.dir_name = 'results'
        self.file_path = file_path
        self.make_results_dir()
        self.df_raw = self.load_data()
        print(f'正在处理 {self.file_path} 文件.....................')
#         print('\n')
        
        self.fc, self.Rc, self.Rt, self.Cp, self.ofc = self.equivalent_circuit_fitting()
        
    # 加载初始数据
    def load_data(self):
        file_type = self.file_path.split('.')[-1]
        if file_type == 'xlsx':
            data_df = pd.read_excel(self.file_path)
        elif file_type == 'csv':
            data_df = pd.read_csv(self.file_path)
        else:
            raise ValueError("文件格式 .{} 不支持!".format(file_type))

        data_df.sort_values("FREQUENCY(Hz)",inplace=True)

        return data_df
    
    # 检测当前文件夹下results问价夹是否存在, 若不存在则创建
    # results用来保存所有的输出图片及结果, 也可自定义名称
    def make_results_dir(self):
        if not os.path.exists(self.dir_name):
            os.makedirs(self.dir_name)
    
    # 选择特定频率范围阻抗画图
    def select_target_impedance(self, plot_figure = False, begin_freq = 10000, end_freq = 2000000):
        mask = (self.df_raw['FREQUENCY(Hz)'] >= begin_freq) & (self.df_raw['FREQUENCY(Hz)'] <= end_freq)
        df = self.df_raw[mask]
        if plot_figure:
            plt.rcParams['figure.dpi'] = 128
            plt.figure(figsize = (3,3.8))
            plt.xlabel("Z' (Ω)")
            plt.ylabel("Z'' (Ω)")
            plt.scatter(df['Rs'],abs(df['X']),marker='s',s=8,color='blue',label='Experimental data')
            plt.legend()
            plt.tight_layout()
            plt.savefig(f'{self.dir_name}/特定区间内的阻抗实部和虚部.jpg',dpi=300)
        return df
    
    # Nyquist图进行等效电路拟合
    def equivalent_circuit_fitting(self):
        df = self.load_data()
        df.drop(['No.'],inplace=True,axis=1)
        df.to_csv('temple.csv',index=None)
        frequencies, Z = preprocessing.readCSV('./temple.csv')
        circuit = 'R1-p(R2,C1)'
        #根据等效电路，从左到右对每个电路元件给出起始的拟合值
        initial_guess = [40, 250, 1e-9]
        circuit = CustomCircuit(circuit, initial_guess=initial_guess)
        circuit.fit(frequencies[1:], Z[1:])
        circuit.save('init_Rt_Cp.json')
        
        ### 用全部数据更新初始猜测值
        with open('init_Rt_Cp.json','r') as f:
            data = json.load(f)
        os.remove('./init_Rt_Cp.json')
        Rc, Rt, Cp = data['Parameters']
        initial_guess = [Rc, Rt, Cp]
        
        # 选定区域拟合
        circuit = 'R1-p(R2,C1)'
#         df = self.load_data()
        df = self.select_target_impedance()
        df.drop(['No.'],inplace=True,axis=1)
        df.to_csv('temple.csv',index=None)
        frequencies, Z = preprocessing.readCSV('./temple.csv')
        circuit = CustomCircuit(circuit, initial_guess=initial_guess)
        circuit.fit(frequencies[1:], Z[1:])
        os.remove('./temple.csv')
        circuit.save('init_Rt_Cp.json')
        with open('init_Rt_Cp.json','r') as f:
            data = json.load(f)
        os.remove('./init_Rt_Cp.json')
        Rc, Rt, Cp = data['Parameters']
        print('-----电学原件的拟合值为:-----')
        print(f'Rc = {Rc} Ohm')
        print(f'Rt = {Rt} Ohm')
        print(f'Cp = {Cp} F')
        
        fc = 1/(2*np.pi*Rt*Cp)*1.1
#         print('\n')
        print('-----临界频率的计算值为:-----')
        print(f'{int(fc)/1000000} MHz')
        with open(f"{self.dir_name}/{self.file_path.split('.')[0]}_output.txt",'w') as f:
            f.write(f'-----电学原件的拟合值为:-----')
            f.write(f'\n')
            f.write(f'Rc = {Rc} Ohm')
            f.write(f'\n')
            f.write(f'Rt = {Rt} Ohm')
            f.write(f'\n')
            f.write(f'Cp = {Cp} F')
            f.write(f'\n')
            f.write(f'-----临界频率的计算值为:-----')
            f.write(f'\n')
            f.write(f'fc = {int(fc)/1000000} MHz')
            f.write(f'\n')
        
        return fc, Rc, Rt, Cp, int(fc)/1000000
        
    # 计算并修正复阻抗模量
    def calc_complex_impedence(self):
        Zi = np.sqrt(self.df_raw['Rs']**2 + self.df_raw['X']**2)
        # 每个频率下矫正
        max_freq_ind = list(self.df_raw['FREQUENCY(Hz)']).index(max(self.df_raw['FREQUENCY(Hz)']))
        Zmax = Zi[max_freq_ind]
        Zci = Zi - Zmax
        start_ind = self.find_start_freq_index()
        end_ind = -1

        return np.array(Zci[start_ind:end_ind]), np.array(self.df_raw['FREQUENCY(Hz)'])[start_ind:end_ind]

    # 找到最近接临界频率的点
    def find_start_freq_index(self):
        freq = list(self.df_raw['FREQUENCY(Hz)'])
        for index in range(len(freq)-1):
            if self.fc > freq[index] and self.fc < freq[index+1]:
                return index if abs(self.fc-freq[index]) < abs(self.fc-freq[index+1]) else index + 1

    # 反傅立叶及傅立叶变换
    def fft(self):
        Zci,freq = self.calc_complex_impedence()

        # 逆傅立叶变化后的阻抗
        Z_irfft = irfft(Zci, n = len(Zci))
        # 逆傅立叶变化后的时间
        T_irfft = irfft(freq,n=len(freq))
    
        # 在做傅立叶变换
        Z_irfft_fft = abs(fft(Z_irfft, n = len(Z_irfft)))
        T_irfft_fft = abs(fft(T_irfft,n=len(T_irfft)))
        
        return np.log10(T_irfft_fft), np.log10(Z_irfft_fft**2)
    
    def plot_linear_reg_line(self, plot_figure = False):
        x,y = self.fft()
        min_x = min(x)
        max_x = max(x)
        model = LinearRegression()
        model.fit(x.reshape(-1, 1),y.reshape(-1, 1))
        k = round(model.coef_[0][0],3)
        if plot_figure:
            plt.figure(figsize = (5,4))
            plt.text(min(x),min(y),f'斜率k = {k}',fontproperties="SimHei")

            min_x_pred = model.predict(np.array([min_x]).reshape(-1, 1))[0][0]
            max_x_pred = model.predict(np.array([max_x]).reshape(-1, 1))[0][0]
            plt.plot([min_x,max_x],[min_x_pred,max_x_pred],'r--')


            plt.scatter(x,y,color='k',marker='s',s=12)
            plt.plot([min(x),max(x)],[min_x_pred, max_x_pred],'r--')
            plt.ylabel('阻抗信号振幅平方的对数', fontproperties="SimHei")
            plt.xlabel('频率的对数', fontproperties="SimHei")
            plt.tight_layout()
            plt.savefig(f'{self.dir_name}/线性拟合曲线.jpg',dpi=300)
        
#         print('\n')
        print('-----线性回归斜率k的值为:-----')
        print(k)
        
#         print('\n')
        print('-----谱维数ds的计算值为:-----')
        ds = (k+5)/2
        print(ds)
        
#         print('\n')
        print('-----ds/fc的值为:-----')
        print(round(ds/self.fc*1000000,3))
        
        with open(f"{self.dir_name}/{self.file_path.split('.')[0]}_output.txt",'a') as f:
            f.write(f'-----线性回归斜率k的值为:-----')
            f.write(f'\n')
            f.write(f'{k}')
            f.write(f'\n')
            
            f.write(f'-----谱维数ds的计算值为:-----')
            f.write(f'\n')
            f.write(f'{ds}')
            f.write(f'\n')
            
            f.write(f'-----ds/fc的值为:-----')
            f.write(f'\n')
            f.write(f'{round(ds/self.fc*1000000,3)}')

        return k, ds, round(ds/self.fc*1000000,3)
    
    def combine_csv(self):
#         os.chdir(self.dir_name)
        path = self.dir_name
        file_list = []
        for file in [i for i in os.listdir(path) if i.endswith('xlsx')]:
#             print(file)
            df = pd.read_excel(path +'/'+ file)
            file_list.append(df)

        result = pd.concat(file_list)   # 合并文件
        result.to_excel(path +'/'+ 'total.xlsx', index=None)  # 保存合并后的文件

        
    def do_analysis(self):
        Zci, freq = self.calc_complex_impedence()
        self.fft()
        k ,ds, ds_fc = self.plot_linear_reg_line()
        df_output = pd.DataFrame(columns = ['文件名','变量名', '变量值'], \
                                 data = zip([self.file_path]*7,['Rc','Rt','Cp','fc','k','ds','ds/fc' ],\
                                            [self.Rc, self.Rt, self.Cp, self.ofc, k ,ds, ds_fc],\
                                           ))
        df_output.to_excel(f"{self.dir_name}/{self.file_path.split('.')[0]}_output.xlsx", index=None)
        print('\n')

In [4]:
for f in [i for i in os.listdir() if i.endswith('csv')]:
    try:
        i = Impedance(f)
        i.do_analysis()
    except:
        print(f'请仔细检查{f}文件的输入')
i.combine_csv()

正在处理 5 gkg KMnO4 数据例.csv 文件.....................
-----电学原件的拟合值为:-----
Rc = 56.55950427254049 Ohm
Rt = 269.682073816942 Ohm
Cp = 1.3261749156956776e-09 F
-----临界频率的计算值为:-----
0.489508 MHz
-----线性回归斜率k的值为:-----
-1.921
-----谱维数ds的计算值为:-----
1.5394999999999999
-----ds/fc的值为:-----
3.145


正在处理 4.4 0 A.csv 文件.....................
-----电学原件的拟合值为:-----
Rc = 84.21740560629466 Ohm
Rt = 282.53399632685563 Ohm
Cp = 1.523765982823428e-09 F
-----临界频率的计算值为:-----
0.406652 MHz
-----线性回归斜率k的值为:-----
-1.665
-----谱维数ds的计算值为:-----
1.6675
-----ds/fc的值为:-----
4.101


